Пробуем сделать простое дерево решений и работать с ним

# Задача 1. Сколько заплатить за месторождение?

Вы покупаете месторождение. Известна геологическая оценка извлекаемых запасов:

- p10 - 1 млн тонн нефти
- p50 - 1.1 млн тонн нефти
- p90 - 1.2 млн тонн нефти

Стоимость 1 тонны нефти в запасах – 1 тыс. руб.

Готовы ли купить месторождение за 1.1 млрд.руб? Сколько вы готовы заплатить?

In [1]:
import json

In [43]:
tree = {"name": "Покупать",
        "type": "decision",
        "child_edges":[
                       {"name": "Да",
                        "voi_info":"with_info",
                        "payoff": -100,
                        "child_node": 
                                    {"name": "Успех",
                                     "type": "chance",
                                     "child_edges": [
                                                    {"name": "Да",
                                                    "payoff": 1000,
                                                    "probability": 0.6,
                                                    "child_node": {"type": "terminal"}
                                                    },
                                                    {"name": "Нет",
                                                    "payoff": -10,
                                                    "probability": 0.4,
                                                    "child_node": {"type": "terminal"}
                                                    },
                                                    ]  
                                    }
                        },
                        {"name": "Нет",
                         "voi_info":"without_info",
                         "payoff": 1000,
                         "child_node": {"type": "terminal"}
                        }
                    ]  
        }

In [57]:
def solve_tree(tree, v=0, p=0):
    """
    рекурсивное решение дерева заданного в виде словаря
    """
    
    if tree['type'] == 'terminal':
        # для терминального узла сохраняем в нем итоговые выигрыши и вероятности
        tree['emv'] = 0
        tree['v'] = v
        tree['p'] = p
    else:
        # или 'decision' или 'chance' с дочерними узлами
        tree['emv'] = 0
        if tree['type'] == 'decision':
            tree['emvs'] = {}
            num = 0
        
        for edge in tree['child_edges']:
            # переберем все связи рассматриваемого узла
            if tree['type'] == 'decision':
                edge['id'] = num
                num = num + 1

            # рекурсивно рассчитаем параметры дочерних узлов
            solve_tree(edge['child_node'], edge['payoff']+v, edge['probability'] if 'probability' in edge else 0 * p)

            # рассчитаем emv рассматриваемого узла
            if tree['type'] == 'decision':
                tree['emvs'][edge['id']] = edge['child_node']['emv'] + edge['payoff']
            else:
                tree['emv'] = tree['emv'] + (edge['child_node']['emv'] + edge['payoff']) * edge['probability']

        # для узла решения определим оптимальное - из максимизации EMV
        if tree['type'] == 'decision':
            tree['emv'] = max(tree['emvs'].values())
            for edge in tree['child_edges']:
                if tree['emvs'][edge['id']] == tree['emv']:
                    edge['probability'] = 1
                else:
                    edge['probability'] = 0
            # оценим VOI для узла решения
            emv_wi = 0
            emv_woi = 0
            calc_voi = [False, False]
            for edge in tree['child_edges']:
                if 'voi_info' in edge:
                    if edge['voi_info']=='with_info':
                        emv_wi = tree['emvs'][edge['id']]
                        calc_voi[0] = True
                    if edge['voi_info']=='without_info':
                        emv_woi = tree['emvs'][edge['id']]
                        calc_voi[1] = True
            if calc_voi == [True, True]:
                tree['voi'] = emv_wi - emv_woi
      



def print_tree(tree, level=0):
    if level == 0:
        print('|','-'*level,  f" {tree['type']} : {tree['name'] if 'name' in tree else ''} [ {tree['emv']}]" )
    else:
        if 'child_node' in tree:
            print('|', '-'*level,  
                  f" {tree['name']} ",
                  f" [payoff:{tree['payoff']} { (' p = ' + str(tree['probability'])) if 'probability' in tree else '' }] -> ",
                  f"  {tree['child_node']['type']} : {tree['child_node']['name'] if 'name' in tree['child_node'] else ''} [ emv = {tree['child_node']['emv']}]",
                  f" ({tree['child_node']['p']} {tree['child_node']['v']})" if tree['child_node']['type']=='terminal' else '' )


    if 'child_edges' in tree:
        for edge in tree['child_edges']:
            print_tree(edge, level=level+2)
            print_tree(edge['child_node'], level=level+2)

solve_tree(tree)
print('===========================')
print_tree(tree)

|   decision : Покупать [ 1000]
| --  Да   [payoff:-100  p = 0] ->    chance : Успех [ emv = 596.0] 
| ----  Да   [payoff:1000  p = 0.6] ->    terminal :  [ emv = 0]  (0.6 900)
| ----  Нет   [payoff:-10  p = 0.4] ->    terminal :  [ emv = 0]  (0.4 -110)
| --  Нет   [payoff:1000  p = 1] ->    terminal :  [ emv = 0]  (1 1000)


In [58]:
tree

{'name': 'Покупать',
 'type': 'decision',
 'child_edges': [{'name': 'Да',
   'voi_info': 'with_info',
   'payoff': -100,
   'child_node': {'name': 'Успех',
    'type': 'chance',
    'child_edges': [{'name': 'Да',
      'payoff': 1000,
      'probability': 0.6,
      'child_node': {'type': 'terminal', 'emv': 0, 'v': 900, 'p': 0.6}},
     {'name': 'Нет',
      'payoff': -10,
      'probability': 0.4,
      'child_node': {'type': 'terminal', 'emv': 0, 'v': -110, 'p': 0.4}}],
    'emv': 596.0},
   'id': 0,
   'probability': 0},
  {'name': 'Нет',
   'voi_info': 'without_info',
   'payoff': 1000,
   'child_node': {'type': 'terminal', 'emv': 0, 'v': 1000, 'p': 1},
   'id': 1,
   'probability': 1}],
 'emv': 1000,
 'emvs': {0: 496.0, 1: 1000},
 'voi': -504.0}

In [34]:
tree2 = {"name": "Покупать",
        "type": "decision",
        "child_edges":[
                        {"name": "Да",
                         "payoff": -100,
                         "child_node": 
                                {"name": "Успех",
                                 "type": "chance",
                                 "child_edges": [
                                                    {"name": "Да",
                                                    "payoff": 100,
                                                    "probability": 0.6,
                                                    "child_node": 
                                                                    {"name": "Результат",
                                                                     "type": "chance",
                                                                     "child_edges": [
                                                                                        {"name": "Да",
                                                                                        "payoff": 1000,
                                                                                        "probability": 0.8,
                                                                                        "child_node": {"type": "terminal"}
                                                                                        },
                                                                                        {"name": "Нет",
                                                                                        "payoff": -10,
                                                                                        "probability": 0.2,
                                                                                        "child_node": {"type": "terminal"}
                                                                                        },
                                                                                    ]  
                                                                    }
                                                                
                                                     
                                                    },
                                                    {"name": "Нет",
                                                    "payoff": -10,
                                                    "probability": 0.4,
                                                    "child_node": {"type": "terminal"}
                                                    },
                                                ]  
                                }
                        },
                        {"name": "Нет",
                         "payoff": -100,
                         "child_node": {"type": "terminal"}
                        }
                    ]  
        }

In [35]:
solve_tree(tree2)
print('===========================')
print_tree(tree2)

|   decision : Покупать [ 434.79999999999995]
| --  Да   [payoff:-100  p = 1] ->    chance : Успех [ emv = 534.8] 
| ----  Да   [payoff:100  p = 0.6] ->    chance : Результат [ emv = 798.0] 
| ------  Да   [payoff:1000  p = 0.8] ->    terminal :  [ emv = 0]  (0.8 1000)
| ------  Нет   [payoff:-10  p = 0.2] ->    terminal :  [ emv = 0]  (0.2 -10)
| ----  Нет   [payoff:-10  p = 0.4] ->    terminal :  [ emv = 0]  (0.4 -110)
| --  Нет   [payoff:-100  p = 0] ->    terminal :  [ emv = 0]  (0 -100)


In [36]:
tree2

{'name': 'Покупать',
 'type': 'decision',
 'child_edges': [{'name': 'Да',
   'payoff': -100,
   'child_node': {'name': 'Успех',
    'type': 'chance',
    'child_edges': [{'name': 'Да',
      'payoff': 100,
      'probability': 0.6,
      'child_node': {'name': 'Результат',
       'type': 'chance',
       'child_edges': [{'name': 'Да',
         'payoff': 1000,
         'probability': 0.8,
         'child_node': {'type': 'terminal', 'emv': 0, 'v': 1000, 'p': 0.8}},
        {'name': 'Нет',
         'payoff': -10,
         'probability': 0.2,
         'child_node': {'type': 'terminal', 'emv': 0, 'v': -10, 'p': 0.2}}],
       'emv': 798.0}},
     {'name': 'Нет',
      'payoff': -10,
      'probability': 0.4,
      'child_node': {'type': 'terminal', 'emv': 0, 'v': -110, 'p': 0.4}}],
    'emv': 534.8},
   'id': 0,
   'probability': 1},
  {'name': 'Нет',
   'payoff': -100,
   'child_node': {'type': 'terminal', 'emv': 0, 'v': -100, 'p': 0},
   'id': 1,
   'probability': 0}],
 'emv': 434.7999999

In [24]:
d = {1:2, 3:4}


In [25]:
d

{1: 2, 3: 4}

In [29]:
max(d)

3

In [52]:

calc_voi = [False,False]

In [55]:
calc_voi == [True, False]

True

In [56]:
mult

NameError: name 'mult' is not defined

In [60]:
import mermaid as mmd

In [61]:
mmd.__version__

'0.6.0'

In [63]:

%%mermaidjs
flowchart LR
   A -- text --> B -- text2 --> C

In [62]:
%%mermaidjs
mindmap
  root((mindmap))
    Origins
      Long history
      ::icon(fa fa-barcode)
      Popularisation
        British popular psychology author Tony Buzan
    Research
      On effectivness<br/>and features
      On Automatic creation
        Uses
            Creative techniques
            Strategic planning
            Argument mapping
    Tools
      Pen and paper
      Mermaid